In [294]:
import pprint
from tatsu import parse

In [308]:
GRAMMAR = '''
    start = expression $ ;

    expression
        =
        | expression expOperator term
        | term
        | trash expression
        ;

    trash = /\w+/;
    
    term
        =
        | term termOperator factor
        | factor
        | trash term
        ;

    factor
        = number
        ;

    number 
        = /\d+/ 
        ;
    
    expOperator
        =
        | plusOperator
        | minusOperator
        | trash expOperator
        ;
        
    termOperator
        =
        | multipleOperator
        | divisionOperator
        | trash termOperator
        ;
        
    plusOperator
        =
        | 'plus'
        | 'dodać'
        ;
    
    minusOperator
        =
        | 'minus'
        | 'odjąć'
        ;
        
    multipleOperator
        = 'razy'
        ;
    
    divisionOperator
        =
        | 'przez'
        | 'dzielone przez'
        | 'nad'
        ;
        
'''

In [309]:
expression = 'policz proszę 5 razy 3 i dodać 88 razy 2'

parsedExpression = parse(GRAMMAR, expression)
print('# PPRINT')
pprint.pprint(parsedExpression, indent=2, width=20)
print()

# PPRINT
[ 'policz',
  [ 'proszę',
    [ [ '5',
        'razy',
        '3'],
      [ 'i',
        [ 'dodać',
          [ '88',
            'razy']]],
      '2']]]



In [310]:
operatorMap = {
    "plus": "+",
    "dodać": "+",
    "minus": "-",
    "odjąć": "-",
    "razy": "*",
    "przez": "/",
    "dzielone przez": "/"
}

In [311]:
def process(e):
    if e in ['+', '-', '*', '/']:
        return e
    if e.isdigit():
        return e
    if e in operatorMap.keys():
        return operatorMap.get(e)
    else:
        return ''

In [312]:
def evaluate(e):
    result = ''
    print('evaluate:', e)
    for subexpression in e:
        if type(subexpression) is not str:
            partial = evaluate(subexpression)
            result += partial
        else:
            result += process(subexpression)
    return result
        
        
evaluate(parsedExpression)

evaluate: ['policz', ['proszę', [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']]]
evaluate: ['proszę', [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']]
evaluate: [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']
evaluate: ['5', 'razy', '3']
evaluate: ['i', ['dodać', ['88', 'razy']]]
evaluate: ['dodać', ['88', 'razy']]
evaluate: ['88', 'razy']


'5*3+88*2'

In [313]:
eval(evaluate(parsedExpression))

evaluate: ['policz', ['proszę', [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']]]
evaluate: ['proszę', [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']]
evaluate: [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']
evaluate: ['5', 'razy', '3']
evaluate: ['i', ['dodać', ['88', 'razy']]]
evaluate: ['dodać', ['88', 'razy']]
evaluate: ['88', 'razy']


191